In [21]:
import csv
import pandas as pd
import numpy as np
import edward as ed
import re
import tensorflow as tf
import math
from sklearn.preprocessing import scale as varscale
from sklearn.model_selection import LeaveOneOut

In [ ]:
def rbf_fun(X, X2=None, lengthscale=1.0, variance=1.0):
    lengthscale = tf.constant(lengthscale, dtype = tf.float32)
    variance = tf.constant(variance, dtype = tf.float32)
    
    X = tf.convert_to_tensor(X,tf.float32)
    Xs = tf.reduce_sum(tf.square(X), 1)
    if X2 is None:
        X2 = X
        X2s = Xs
    else:
        X2 = tf.convert_to_tensor(X2,tf.float32)
        X2s = tf.reduce_sum(tf.square(X2), 1)
        
    square = tf.reshape(Xs, [-1, 1]) + tf.reshape(X2s, [1, -1]) - \
    tf.multiply(2.0 , tf.matmul(X, X2, transpose_b=True))
    
    output = variance * tf.exp(-tf.cast(square,dtype=tf.float32) / tf.multiply(tf.constant(2.0, dtype = tf.float32),tf.square(lengthscale)))
    grad_lengthscale = tf.multiply(output, (-2.0)*square/(lengthscale**3.0))
    grad_variance = tf.multiply(2.0*tf.sqrt(variance), tf.exp(-tf.cast(square,dtype=tf.float32) / tf.multiply(tf.constant(2.0, dtype = tf.float32),tf.square(lengthscale))))
    return output, grad_lengthscale, grad_variance

def matern_fun(X, X2=None, lengthscale_in=1.0, gamma_in=3/2):
    lengthscale = tf.constant(lengthscale_in, dtype = 'float32')
    gamma = tf.constant(gamma_in, dtype = 'float32')
    
    X = tf.convert_to_tensor(X,tf.float32)
    Xs = tf.reduce_sum(tf.square(X), 1)
    if X2 is None:
        X2 = X
        X2s = Xs
    else:
        X2 = tf.convert_to_tensor(X2,tf.float32)
        X2s = tf.reduce_sum(tf.square(X2), 1)
    
    square = tf.reshape(Xs, [-1, 1]) + tf.reshape(X2s, [1, -1]) - \
    2 * tf.matmul(X, X2, transpose_b=True)
        
    temp_sess = tf.Session()
    if gamma_in == 3/2:
        output = (1.0 + (tf.sqrt(3.0)*tf.sqrt(square))/lengthscale) * \
    tf.exp(-(tf.sqrt(3.0)*tf.sqrt(square))/lengthscale)
        grad = tf.gradients(ys=output, xs = [lengthscale])
        return output, grad
    
    if gamma_in == 5/2:
        output = (1.0 + tf.sqrt(5.0)*tf.sqrt(square)/lengthscale + \
                5.0*square/(3.0*lengthscale**2)) * tf.exp(-(tf.sqrt(5.0)*tf.sqrt(square)/lengthscale))
        grad = tf.gradients(ys=output, xs = lengthscale)
        print(temp_sess.run(output))
        print(temp_sess.run(grad))
        return output, grad
    
def pred_f_and_cov(X_input, y_input, X_star_input, fun_form, sigma_sq_n, param_list):
    X_input = tf.cast(X_input, tf.float32)
    y_input = tf.reshape(tf.cast(y_input, tf.float32), [-1,1])
    X_star_input = tf.reshape(tf.cast(X_star_input, tf.float32),[1,-1])
    
    if fun_form == 'rbf':
        lengthscale_in = param_list[0]
        variance_in = param_list[1]
        
        test_sess = tf.Session()
        
        f_new = tf.matmul(rbf_fun(X_star_input, X_input, lengthscale_in, variance_in)[0], \
                          tf.linalg.inv(rbf_fun(X = X_input, lengthscale = lengthscale_in, variance = variance_in)[0] +\
                          tf.multiply(sigma_sq_n, tf.eye(int(X_input.shape[0])))))
        f_new = tf.matmul(f_new,y_input)
        cov_new = rbf_fun(X_star_input,lengthscale = lengthscale_in, variance = variance_in)[0] - \
        tf.matmul(rbf_fun(X_star_input,X_input, lengthscale_in, variance_in)[0],\
                  tf.matmul(tf.linalg.inv(rbf_fun(X_input, lengthscale = lengthscale_in, variance = variance_in)[0] + \
                                           tf.multiply(sigma_sq_n, tf.eye(int(X_input.shape[0])))),\
                            rbf_fun(X_input,X_star_input, lengthscale_in, variance_in)[0]))
    else:
        lengthscale_in = param_list[0]
        gamma_in = param_list[1]
        f_new = tf.matmul(matern_fun(X_star_input, X_input, lengthscale_in, gamma_in)[0], \
                          tf.linalg.inv(metern_fun(X = X_star_input, lengthscale_in = lengthscale_in, gamma_in = gamma_in)[0] + \
                          tf.multiply(sigma_sq_n, tf.eye(int(X_star_input.shape[0])))))
        f_new = tf.matmul(f_new,y_input)
        cov_new = matern_fun(X_star_input,lengthscale_in = lengthscale_in, gamma_in = gamma_in)[0] -\
        tf.matmul(matern_fun(X_star_input,X_input, lengthscale_in, gamma_in)[0],\
                  tf.matmul(tf.linalg.inv(matern_fun(X_star_input, lengthscale = lengthscale_in, gamma = gamma_in)[0] + \
                                           tf.multiply(sigma_sq_n, tf.eye(int(X_star_input.shape[0])))),\
                            matern_fun(X_input,X_star_input, lengthscale_in, gamma_in)[0]))
    
    return f_new, cov_new
    
def gp_loocv(X_input, y_input, fun_form, sigma_sq_n, param_list):
    mu_all = np.zeros(X_input.shape[0])
    cov_all = np.zeros(X_input.shape[0])
    y_input_tensor = tf.cast(y_input, tf.float32)
    temp_sess_loocv = tf.Session()
    loo = LeaveOneOut()
    
    for train_index, test_index in loo.split(X_input):
        X_star_input = tf.reshape(X_input[test_index,:],(1,-1))
        X_other_input = X_input[train_index,:]
        y_other_input = y_input[train_index]
        f_new, cov_new = pred_f_and_cov(X_other_input, y_other_input, X_star_input, fun_form, sigma_sq_n, param_list)
        mu_all[test_index] = temp_sess_loocv.run(f_new)
        cov_all[test_index] = temp_sess_loocv.run(cov_new)
        print(temp_sess_loocv.run(f_new))
    
    mu_all_tensor = tf.convert_to_tensor(mu_all,dtype=tf.float32)
    cov_all_tensor = tf.convert_to_tensor(cov_all,dtype=tf.float32)
    
    log_prob = tf.reduce_sum(tf.constant(-1.0/2.0)*tf.log(tf.square(cov_all_tensor))) -\
    tf.reduce_sum(tf.divide(tf.square(tf.subtract(y_input_tensor, mu_all_tensor)),(tf.constant(2.0)*tf.square(cov_new)))) - \
    tf.constant(1.0/2.0)*tf.log(tf.constant(2.0)*tf.constant(math.pi, dtype = tf.float32))*X_input.shape[0]
    
    log_prob_val = temp_sess_loocv.run(log_prob)
    
    return log_prob

def diff_loocv_each(K_mat, y_vec, diff_k_theta_in):
    y_vec = tf.reshape(tf.cast(y_vec, tf.float32), [-1,1])
    K_mat_inv = tf.linalg.inv(K_mat)
    alpha = tf.matmul(K_mat_inv, y_vec)
    
    z_j = tf.matmul(K_mat_inv,diff_k_theta_in)
    diff_l_theta = -tf.multiply(tf.matrix_diag_part(K_mat_inv), tf.multiply(alpha,tf.matmul(z_j,alpha)) - \
                (1.0/2.0)*tf.multiply(1.0 + tf.divide(tf.square(alpha),tf.matrix_diag_part(K_mat_inv)), tf.matrix_diag_part(tf.matmul(z_j, K_mat_inv))))
    diff_out = tf.reduce_sum(diff_l_theta)
    temp_sess = tf.Session()
    output_grad = temp_sess.run(diff_out) 
        
    return output_grad

def grad_descent(X_input, y_input, fun_form, param_list_in, learning_rate, max_iter):
    if fun_form == 'rbf':
        for n in range(max_iter):
            K_mat, grad_length, grad_var = rbf_fun(X_input,lengthscale = param_list_in[0], variance = param_list_in[1])

            diff_param_length = diff_loocv_each(K_mat, y_input, grad_length)
            diff_param_var = diff_loocv_each(K_mat,y_input,grad_var)
            
            param_list_in[0] = param_list_in[0] - (diff_param_length * learning_rate/math.sqrt(n+1))
            param_list_in[1] = param_list_in[1] - (diff_param_var * learning_rate/math.sqrt(n+1))
            
            print(diff_param_length)
            print(diff_param_var)
            
    return param_list_in

In [78]:
gp_loocv(x_train_scaled, y_train_scaled, 'rbf', 0.000001, [1,1])

[[-8.750282e-18]]
[[-1.5702258e-24]]
[[-1.4053622e-17]]
[[3.1092536e-24]]
[[-5.4921034e-29]]
[[-3.1123084e-15]]
[[-4.1167574e-24]]


KeyboardInterrupt: 

In [55]:
k_test, grad_len, grad_var = rbf_fun(x_train_scaled, variance = 15)

In [81]:
grad_descent(x_train_scaled, y_train_scaled, 'rbf',[1,1], 0.00003,1000)

-1.4495151e-07
-39248.88
-1.403996e-08
-572.89886
-1.38074885e-08
-541.5746
-1.3631535e-08
-518.115
-1.3488034e-08
-499.11963
-1.3366025e-08
-483.118
-1.3259398e-08
-469.2005
-1.3164475e-08
-456.8846
-1.30788e-08
-445.8354
-1.3000739e-08
-435.79657
-1.2928854e-08
-426.58505
-1.2862133e-08
-418.0978
-1.2800049e-08
-410.2019
-1.27418875e-08
-402.85397
-1.2687187e-08
-395.94247
-1.2635494e-08
-389.46228
-1.2586502e-08
-383.32004
-1.2539935e-08
-377.50415
-1.2495702e-08
-371.98068
-1.2453415e-08
-366.7237
-1.241294e-08
-361.71613
-1.2374192e-08
-356.92688
-1.2336924e-08
-352.3276
-1.2301066e-08
-347.92285
-1.2266611e-08
-343.6957
-1.2233375e-08
-339.5977
-1.2201253e-08
-335.68008
-1.21701955e-08
-331.89322
-1.2140151e-08
-328.23032
-1.2111065e-08
-324.70657
-1.2082856e-08
-321.28036
-1.2055562e-08
-317.95947
-1.2028959e-08
-314.74728
-1.2003087e-08
-311.6315
-1.1977939e-08
-308.61435
-1.19534835e-08
-305.66565
-1.1929679e-08
-302.81125
-1.1906484e-08
-300.01825
-1.1883763e-08
-297.3211
-1.

KeyboardInterrupt: 

In [33]:
test = temp_sess.run(grad_var)

In [37]:
grad_ascent(x_)

array([2.08808709e-17, 0.00000000e+00, 2.00000000e+00, 0.00000000e+00,
       5.12397956e-38, 7.59243536e-34, 9.75949953e-31, 0.00000000e+00,
       2.70193532e-19, 6.70224853e-25, 7.04466544e-25, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.06690903e-33,
       0.00000000e+00, 9.55244546e-34, 5.86867778e-20, 1.36780132e-35,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.98500509e-32, 4.13884472e-20, 2.56009210e-35,
       4.19628642e-26, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.10182148e-35, 0.00000000e+00, 0.00000000e+00, 9.73428047e-33,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.16722443e-18, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.26033449e-26, 0.00000000e+00, 0.00000000e+00,
       3.32171524e-33, 8.71465774e-22, 0.00000000e+00, 0.00000000e+00,
      

In [13]:
temp_sess = tf.Session()

In [5]:
#Data importation
year_vec = np.array(range(1856,2019))
dict_all_data = dict()
iter_year = 0
dat_each_year = list()

with open('ssta_dat.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        indicator = row[0].strip()
        if not re.match('May', indicator):
            each_dat = row[1:len(row)]
            for i in range(len(each_dat)):
                each_dat[i] = float(each_dat[i].strip())
            dat_each_year.extend(each_dat)
        else:
            if len(dat_each_year) != 0:
                dict_all_data[year_vec[iter_year]] = np.array(dat_each_year)
                dat_each_year = list()
                iter_year += 1
    dict_all_data[year_vec[iter_year]] = np.array(dat_each_year)

In [6]:
dat_out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_all_data.items() ]))
dat_out_final = dat_out.T
dat_out_final = dat_out_final.drop(dat_out_final.columns[dat_out_final.apply(lambda col: col.mean() > 100)], axis=1)

In [7]:
y_val_dict = dict()
filepath = 'newlandfreq.txt'  
with open(filepath) as fp:  
    each_line = fp.readline()
    while each_line:
        temp_split = each_line.split()
        for i in range(len(temp_split)):
            temp_split[i] = float(temp_split[i])
        y_val_dict[temp_split[0]] = np.array(temp_split[1:len(temp_split)])
        each_line = fp.readline()

In [8]:
y_out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in y_val_dict.items() ]))
y_out_final = y_out.T
y_out_final = y_out_final.loc[1856:,].reset_index()
dat_out_final = dat_out_final.loc[:2016,].reset_index()
y_train_full = y_out_final.iloc[:,1]
y_train_full = y_train_full.to_frame().reset_index()

dat_out_final = dat_out_final.values
y_train_full = y_train_full.iloc[:,1].values

In [9]:
N = dat_out_final.shape[0]
D = dat_out_final.shape[1]

In [10]:
y_train_scaled = varscale(y_train_full, with_std = False)
x_train_scaled = varscale(dat_out_final)

In [ ]:
x_train_scaled = tf.cast(x_train_scaled, tf.float32)
y_train_scaled = tf.reshape(tf.cast(y_train_scaled, tf.float32), [-1,1])

In [ ]:
from edward.models import MultivariateNormalTriL,Normal
from edward.util import rbf

X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalTriL(loc=tf.zeros(N), scale_tril=tf.cholesky(rbf(X)))
y = Poisson(rate= tf.nn.softplus(f))

In [ ]:
qf = Normal(loc = tf.get_variable('qf/loc',[N]), scale = tf.nn.softplus(tf.get_variable('qf/scale',N)))

In [ ]:
test = tf.cholesky(rbf(dat_out_final.astype('float32'))).eval()
test[8]

In [ ]:
inference_vi = ed.KLqp({f:qf}, data={X:x_train_scaled, y:y_train_scaled})
inference_vi.run(n_iter=5000)

In [ ]:
qmu = Empirical(tf.Variable(tf.zeros(1000)))
inference_hmc = ed.HMC({f: qf}, data={X:dat_out_final, y:y_train_full})

In [ ]:
y_post = ed.copy(y, {f: qf})

In [ ]:
y_post.eval()

In [ ]:
each_error = ed.evaluate('mean_squared_error', data={X: x_train_scaled, y_post: y_train_scaled})
each_error

In [ ]:
from sklearn.model_selection import KFold
kf = kFold(n_splits = 5)


In [ ]:
X1 = tf.placeholder(tf.float32, [N-1, D])
f1 = MultivariateNormalTriL(loc=tf.zeros(N-1), scale_tril=tf.cholesky(rbf(X1)))
y1 = tf.nn.softplus(f1)

qf1 = Normal(loc = tf.get_variable('qf/loc1',[N-1]), scale = tf.nn.softplus(tf.get_variable('qf/scale1',N-1)))

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loocv_error_out = list()
for train_index, test_index in loo.split(dat_out_final):
    X_train, X_test = dat_out_final[train_index], dat_out_final[test_index]
    y_train, y_test = y_train_full[train_index], y_train_full[test_index]
    inference = ed.KLqp({f1:qf1}, data={X1:X_train, y1:y_train})
    inference.run(n_iter=1000)
    each_error = ed.evaluate('mean_squared_error', data={X1: X_test, y1: y_test})
    loocv_error_out.append(each_error)

In [ ]:
#y_post.eval() - y_train_full
math.sqrt(np.mean((y_post.eval() - y_train_full)**2))
#_train_full

In [ ]:
y_train_full[89]